In [1]:
import pandas as pd
import numpy as np
from eda.tools import seq_to_num, acc_score, prep_submit
from models.baseline import Baseline
from models.pipeline import Pipeline
from models.diff_table import DiffTable
from models.linear_model import LinearModel
from models.Nonlinear_model import NonLinearModel
from models.mark_chain import MarkovChains
from models.lin_reg import LinReg
from models.rnn import RNN
from preproc.filters import rnn_filter, rnn_filterv2

Using TensorFlow backend.


In [2]:
df_test = pd.read_csv('data/kaggle_test.csv', index_col=0)
# trie_predictions = pd.read_csv('submit/trie_predict.csv', index_col=0)
# df_unsolved = df_test[~df_test.index.isin(trie_predictions.index)]
X_val = seq_to_num(df_test.Sequence, target_split=False, pad=False)

In [3]:
models = [
    ('DT', DiffTable(), None),
    ('LRR', LinearModel(slice=16, minlen=14), None),
    ('NLRR', NonLinearModel(), None),
    ('MCS', MarkovChains(6, 20, 4), None),
    ('LinReg', LinReg(), None),
    ('RNN', RNN(), rnn_filterv2)
]

In [4]:
pipe = Pipeline(models, Baseline(), True)

In [5]:
pred = pipe.predict(X_val)

113845it [03:32, 536.71it/s]


solved by DT: 4178
solved by LRR: 8309


/Users/ptyshevs/udata/int_seq/models/Nonlinear_model.py:61: RuntimeWarning: overflow encountered in double_scalars
  return [x1 ** 2, x2 ** 2, x1 * x2, x1, x2]
/Users/ptyshevs/udata/int_seq/models/Nonlinear_model.py:64: RuntimeWarning: overflow encountered in double_scalars
  return sum([x[0] ** 2 * solution[0], x[1] ** 2 * solution[1], x[0] * x[1] * solution[2], x[0] * solution[3],
/Users/ptyshevs/udata/int_seq/models/Nonlinear_model.py:65: RuntimeWarning: overflow encountered in double_scalars
  x[1] * solution[4], solution[5]])


solved by NLRR: 907


0it [00:00, ?it/s]/Users/ptyshevs/envs/loc_env/lib/python3.6/site-packages/sklearn/linear_model/base.py:485: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  linalg.lstsq(X, y)
2it [00:00, 18.17it/s]

solved by MCS: 3878


96573it [17:34, 91.62it/s] 


solved by LinReg: 27393
solved by RNN: 29916
solved by fallback-model Baseline(function=mode): 39264


In [6]:
prep_submit(pred, 'submit_without_tree.csv')